<a href="https://colab.research.google.com/github/mahmud-nobe/Cluster-Membership/blob/master/cluster_membership.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import packages
import pandas as pd
import pandas_profiling as pd_prof
import numpy as np
import matplotlib.pyplot as plt

## Use random forest to create and evaluate new model
from sklearn.ensemble import RandomForestRegressor

In [38]:

# import member dataset
member = pd.read_csv('https://raw.githubusercontent.com/mahmud-nobe/Cluster-Membership/master/n884_mem.csv')
member.describe()

,RA_ICRS,DE_ICRS,Source,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP,PMemb,_RA.icrs,_DE.icrs
count,1107.000000,1107.000000,1.107000e+03,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1107.000000,1098.000000,1107.000000,1107.000000,1107.000000
mean,35.669128,57.136607,4.584198e+17,135.099489,-3.563817,0.394660,0.055318,-0.589288,0.103309,-1.035505,0.115290,266.095754,15.212058,1.032836,0.488889,35.669133,57.136611
std,0.203451,0.112922,2.951795e+13,0.110841,0.112720,0.050145,0.030342,0.217097,0.057488,0.219616,0.062315,23.490328,1.849833,0.361234,0.245931,0.203451,0.112922
min,35.343833,56.821045,4.583058e+17,134.923423,-3.863963,0.283100,0.019500,-1.437000,0.035000,-1.879000,0.042000,129.000000,6.210630,0.303563,0.100000,35.343838,56.821049
25%,35.508695,57.068122,4.584059e+17,135.013822,-3.635055,0.358750,0.034250,-0.712000,0.064000,-1.164500,0.073500,253.000000,14.246502,0.726127,0.300000,35.508699,57.068126
50%,35.624218,57.139308,4.584115e+17,135.076991,-3.565761,0.394100,0.045800,-0.586000,0.085000,-1.049000,0.096000,268.000000,15.608300,1.020511,0.500000,35.624224,57.139313
75%,35.824777,57.209976,4.584531e+17,135.166715,-3.490694,0.431800,0.066300,-0.471500,0.123000,-0.930500,0.138500,282.000000,16.475282,1.306833,0.700000,35.824782,57.209982
max,36.154135,57.398243,4.584645e+17,135.382923,-3.293396,0.507300,0.188400,0.301000,0.396000,-0.087000,0.448000,327.000000,17.996572,2.581698,1.000000,36.154139,57.398246


In [39]:
pd_prof.ProfileReport(member)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,18
Number of observations,1107
Total Missing (%),0.0%
Total size in memory,155.8 KiB
Average record size in memory,144.1 B
Numeric,11
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,7


In [41]:
member = member.dropna()
member.describe()

,RA_ICRS,DE_ICRS,Source,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP,PMemb,_RA.icrs,_DE.icrs
count,1098.000000,1098.000000,1.098000e+03,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000
mean,35.669378,57.136251,4.584197e+17,135.099739,-3.564103,0.394712,0.055176,-0.588641,0.102953,-1.034608,0.114573,266.461749,15.207598,1.032836,0.489891,35.669383,57.136256
std,0.203849,0.113182,2.955938e+13,0.111037,0.113001,0.050184,0.030298,0.216352,0.057047,0.219179,0.061320,22.974856,1.851919,0.361234,0.245873,0.203849,0.113182
min,35.343833,56.821045,4.583058e+17,134.923423,-3.863963,0.283100,0.019500,-1.437000,0.035000,-1.879000,0.042000,129.000000,6.210630,0.303563,0.100000,35.343838,56.821049
25%,35.508529,57.068000,4.584059e+17,135.013743,-3.636024,0.358525,0.034200,-0.711000,0.064000,-1.163000,0.073000,254.000000,14.246426,0.726127,0.300000,35.508533,57.068005
50%,35.624004,57.139031,4.584114e+17,135.077246,-3.566391,0.394100,0.045600,-0.586000,0.085000,-1.049000,0.095500,268.000000,15.604971,1.020511,0.500000,35.624009,57.139035
75%,35.824912,57.209822,4.584531e+17,135.167126,-3.490616,0.431800,0.066175,-0.471250,0.123000,-0.930000,0.138000,282.000000,16.471650,1.306833,0.700000,35.824916,57.209826
max,36.154135,57.398243,4.584645e+17,135.382923,-3.293396,0.507300,0.188400,0.301000,0.396000,-0.087000,0.384000,327.000000,17.996572,2.581698,1.000000,36.154139,57.398246


In [42]:
features = member.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12,13]]
features

,RA_ICRS,DE_ICRS,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP
0,35.414700,56.998612,135.016952,-3.741123,0.4406,0.0304,-0.340,0.061,-0.800,0.066,258,14.762490,0.658170
1,35.417161,56.998057,135.018405,-3.741186,0.4387,0.0980,-0.303,0.190,-0.970,0.197,280,17.157730,1.647366
2,35.392245,56.996236,135.006249,-3.747538,0.4182,0.0435,-0.746,0.084,-1.338,0.093,270,15.710485,0.956811
3,35.388396,56.992285,135.005627,-3.751968,0.3541,0.0242,-0.688,0.055,-1.219,0.053,253,13.630529,0.637017
4,35.409224,57.006802,135.011336,-3.734447,0.3837,0.0590,-0.453,0.107,-1.159,0.116,273,16.240725,1.258944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,35.475146,57.108823,135.010074,-3.626319,0.4078,0.1628,-0.069,0.273,-1.517,0.295,284,17.955290,1.558842
1103,35.820625,57.127543,135.179961,-3.543967,0.4018,0.0393,-0.490,0.070,-0.933,0.081,255,15.145645,0.795093
1104,35.504081,57.113965,135.023094,-3.616097,0.3138,0.0682,-1.096,0.125,-0.913,0.136,245,16.569098,1.390498
1105,35.875661,57.121199,135.210233,-3.539511,0.4552,0.0455,-0.395,0.085,-0.938,0.100,246,15.643492,0.929671


In [43]:
features.describe()

,RA_ICRS,DE_ICRS,GLON,GLAT,plx,e_plx,pmRA,e_pmRA,pmDE,e_pmDE,o_Gmag,Gmag,BP-RP
count,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000,1098.000000
mean,35.669378,57.136251,135.099739,-3.564103,0.394712,0.055176,-0.588641,0.102953,-1.034608,0.114573,266.461749,15.207598,1.032836
std,0.203849,0.113182,0.111037,0.113001,0.050184,0.030298,0.216352,0.057047,0.219179,0.061320,22.974856,1.851919,0.361234
min,35.343833,56.821045,134.923423,-3.863963,0.283100,0.019500,-1.437000,0.035000,-1.879000,0.042000,129.000000,6.210630,0.303563
25%,35.508529,57.068000,135.013743,-3.636024,0.358525,0.034200,-0.711000,0.064000,-1.163000,0.073000,254.000000,14.246426,0.726127
50%,35.624004,57.139031,135.077246,-3.566391,0.394100,0.045600,-0.586000,0.085000,-1.049000,0.095500,268.000000,15.604971,1.020511
75%,35.824912,57.209822,135.167126,-3.490616,0.431800,0.066175,-0.471250,0.123000,-0.930000,0.138000,282.000000,16.471650,1.306833
max,36.154135,57.398243,135.382923,-3.293396,0.507300,0.188400,0.301000,0.396000,-0.087000,0.384000,327.000000,17.996572,2.581698


In [44]:
targets = member['PMemb']
targets

0       0.5
1       0.3
2       0.7
3       0.4
4       0.2
       ... 
1102    0.1
1103    0.6
1104    0.2
1105    0.3
1106    0.6
Name: PMemb, Length: 1098, dtype: float64

In [45]:
member.dtypes

RA_ICRS     float64
DE_ICRS     float64
Source        int64
GLON        float64
GLAT        float64
plx         float64
e_plx       float64
pmRA        float64
e_pmRA      float64
pmDE        float64
e_pmDE      float64
o_Gmag        int64
Gmag        float64
BP-RP       float64
PMemb       float64
Cluster      object
_RA.icrs    float64
_DE.icrs    float64
dtype: object

In [46]:
from numpy import float32
features = features.astype('float32')
targets = targets.astype('float32')
features.dtypes

RA_ICRS    float32
DE_ICRS    float32
GLON       float32
GLAT       float32
plx        float32
e_plx      float32
pmRA       float32
e_pmRA     float32
pmDE       float32
e_pmDE     float32
o_Gmag     float32
Gmag       float32
BP-RP      float32
dtype: object

In [47]:
# Use Random Forest on whole dataset and calculate the R^2 value 
# considering the out of bag (OOB) samples
rfr = RandomForestRegressor(n_estimators = 100, oob_score = True)
rfr.fit(features, targets)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [49]:
rfr.score(features, targets)

0.9444657585409276

In [0]:
import random
def test_train_split(data, test_ratio):
  ntest = int(len(data)*test_ratio)
  ntrain = int(len(data) - ntest)

  ind_test = random.sample(range(len(data)), k = ntest)
  ind_train = list(set(range(len(data))) - set(ind_test))

  test = data.iloc[ind_test]
  train = data.iloc[ind_train]

  return test, train

In [0]:
test_features, train_features = test_train_split(features, 0.3)
test_targets, train_targets = test_train_split(targets, 0.3)

In [66]:
rfr.fit(train_features, train_targets)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=True,
                      random_state=None, verbose=0, warm_start=False)

In [74]:
test_predict = rfr.predict(test_features)
# print(test_predict, test_targets)
rfr.score(test_features, test_targets)

-0.04984541847293156

In [68]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_targets, test_predict))

ValueError: ignored